# Yelp API - Lab


## Introduction 

We've seen how the Yelp API works and how to create basic visualizations using Folium. It's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 

* Using pagination and multiple functions, gather large amounts of data from an API, parse the data and make sense of it with meaningful analysis
* Create maps using Folium

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you but make the request below.

In [40]:
#import libraies used
import requests
import json
import pandas as pd

In [41]:
#function to get api key 
def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [42]:
#import dictionary containing API key
keys = get_keys("/Users/brentsair/.secret/yelp_api.json")

#access key from dictionary
api_key = keys['api_key']

In [50]:
term = 'gluten free'
location = 'Seattle'
SEARCH_LIMIT = 1
offset = 50
url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit':SEARCH_LIMIT,
                'offset':offset
            }
response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text[:1000])

<Response [200]>
<class 'str'>
{"businesses": [{"id": "-FOAQv22SXtSBs7nptI3UA", "alias": "serious-pie-seattle", "name": "Serious Pie", "image_url": "https://s3-media3.fl.yelpcdn.com/bphoto/K5cPEhf5ZLkujN73bBNleQ/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/serious-pie-seattle?adjust_creative=OCjU-NiCrYlVSMLPHg0bdA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=OCjU-NiCrYlVSMLPHg0bdA", "review_count": 4070, "categories": [{"alias": "pizza", "title": "Pizza"}], "rating": 4.0, "coordinates": {"latitude": 47.61296, "longitude": -122.34047}, "transactions": [], "price": "$$", "location": {"address1": "316 Virginia St", "address2": null, "address3": "", "city": "Seattle", "zip_code": "98121", "country": "US", "state": "WA", "display_address": ["316 Virginia St", "Seattle, WA 98121"]}, "phone": "+12068387388", "display_phone": "(206) 838-7388", "distance": 1443.3390452329982}, {"id": "6I28wDuMBR5WLMqfKxaoeg", "alias": "pike-place-chowder-seattle", "name"

## Pagination

Now that you have an initial response, you can examine the contents of the JSON container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [60]:
# Your code here; use a function or loop to retrieve all the results from your original r

1

In [169]:
def Yelp_api(search_limit, num_of_requests, location, offset, term):
    list_of_responses=[]
    for request in range(num_of_requests):
        url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit':search_limit,
                'offset':request*offset
            }
        response = requests.get(url, headers=headers, params=url_params)
        if response.status_code == 200:
            response_dict = response.json()
            list_of_responses.append(response_dict)
            
        else:
            print("Error: unable to retrieve hits.  Server responded with status code", response.status_code)
    return list_of_responses
    
test = Yelp_api(1, 6, 'Seattle', 50, 'gluten free')    
test[0]

{'businesses': [{'id': 'oZQwAojMr5N-VBxPTaE9yQ',
   'alias': 'i-love-my-gff-seattle-3',
   'name': 'I Love My GFF',
   'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/q2TjDCaEyAS7ZTb70zhNjw/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/i-love-my-gff-seattle-3?adjust_creative=OCjU-NiCrYlVSMLPHg0bdA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=OCjU-NiCrYlVSMLPHg0bdA',
   'review_count': 91,
   'categories': [{'alias': 'gluten_free', 'title': 'Gluten-Free'},
    {'alias': 'vegetarian', 'title': 'Vegetarian'},
    {'alias': 'foodstands', 'title': 'Food Stands'}],
   'rating': 4.5,
   'coordinates': {'latitude': 47.6243996044122,
    'longitude': -122.336994657105},
   'transactions': ['pickup', 'delivery'],
   'price': '$',
   'location': {'address1': '592 Terry Ave N',
    'address2': None,
    'address3': '',
    'city': 'Seattle',
    'zip_code': '98109',
    'country': 'US',
    'state': 'WA',
    'display_address': ['592 Terry Ave N'

## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [164]:
# Your code here
# len(list_of_responses)
# df = pd.DataFrame(list_of_responses)

df = pd.DataFrame(test)

df.head()

,0
0,businesses
1,total
2,region
3,businesses
4,total


In [155]:
df_updated = pd.DataFrame([])
for i in range(len(df.businesses)):
    df_updated = df_updated.append(df.businesses[i])
df_updated()

TypeError: 'DataFrame' object is not callable

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [149]:
# Your code here
bee = pd.DataFrame(df.businesses[0])
bee.head()
bee= bee.append(df.businesses[1])
bee.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,oZQwAojMr5N-VBxPTaE9yQ,i-love-my-gff-seattle-3,I Love My GFF,https://s3-media3.fl.yelpcdn.com/bphoto/q2TjDC...,False,https://www.yelp.com/biz/i-love-my-gff-seattle...,91,"[{'alias': 'gluten_free', 'title': 'Gluten-Fre...",4.5,"{'latitude': 47.6243996044122, 'longitude': -1...","[pickup, delivery]",$,"{'address1': '592 Terry Ave N', 'address2': No...",,,158.825130
0,Wm0soYlpuNJ8muQgckmrng,cask-and-trotter-seattle-2,Cask & Trotter,https://s3-media2.fl.yelpcdn.com/bphoto/iEvZd5...,False,https://www.yelp.com/biz/cask-and-trotter-seat...,546,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.0,"{'latitude': 47.626012, 'longitude': -122.3390...",[],$$,"{'address1': '711 Westlake Ave N', 'address2':...",+12064534756,(206) 453-4756,269.951664


## Summary

Nice work! In this lab, you've made multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice interactive map to display the results using Folium! Well done!